In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install beautifulsoup4

In [3]:
!pip install chardet

In [4]:
!pip install nltk

In [5]:
pip install summa


Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk import pos_tag
import networkx as nx
import matplotlib.pyplot as plt


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from bs4 import BeautifulSoup

def extract_text_from_html(file_path, element=None):
    try:
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            soup = BeautifulSoup(file, 'html.parser')
            if element:
                texts = [elem.get_text(separator=' ', strip=True) for elem in soup.find_all(element)]
                text = ' '.join(texts)
            else:
                text = soup.get_text(separator=' ', strip=True)
        return text
    except Exception as e:
        print(f"Error extracting text from {file_path}: {e}")
        return ""


In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import re

# Ensure necessary NLTK resources are downloaded
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('words')  # Download the words corpus

# Load the set of English words from NLTK
english_words = set(words.words())

def get_wordnet_pos(treebank_tag):
    """
    Convert treebank POS tags to WordNet POS tags.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize into words
    words = word_tokenize(text)
    
    # Lowercase
    words = [word.lower() for word in words]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    # Add custom words to the stop words set
    custom_stop_words = ['include', 'u', 'de']
    stop_words.update(custom_stop_words)
    words = [word for word in words if word not in stop_words and word.isalpha()]
    
    # Remove words that are not in the English dictionary
    words = [word for word in words if word in english_words]
    
    # POS tagging
    tagged_words = pos_tag(words)
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words_lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_words]
    words_lemmatize = [word for word in words_lemmatized if word not in stop_words and word.isalpha()]
    
    return words_lemmatize



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [9]:
def remove_duplicates_preserve_order(words):
    unique_words = []
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
    return unique_words

In [10]:
def save_text(text, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)

In [11]:
import os

# Make sure to define or import your `extract_text_from_html`, `preprocess_text`, `remove_duplicates_preserve_order`, and `save_sentences` functions here

def process_transcripts(root_folder, output_root):
    english_words_set = set(nltk.corpus.words.words())
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                
                # Extract text from HTML
                text = extract_text_from_html(file_path)
                
                # Preprocess the entire text
                processed_words = preprocess_text(text)  # Preprocess the entire text at once
                
                # Remove duplicates from the list of words
                unique_words = remove_duplicates_preserve_order(processed_words)  # Remove duplicates
                
                filtered_words = [word for word in unique_words if word in english_words_set]
                
                # Reconstruct the text from unique words
                processed_text = ' '.join(filtered_words)  # Reconstruct the text
                
                # Define output directory and file path
                relative_path = os.path.relpath(root, root_folder)
                output_dir = os.path.join(output_root, relative_path)
                os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
                output_file = os.path.splitext(file)[0] + '.txt'
                output_path = os.path.join(output_dir, output_file)
                
                # Save processed text to file
                save_text(processed_text, output_path)



In [12]:
root_folder = 'Data'
output_root = 'Sentences'
if not os.path.exists(output_root):
    os.makedirs(output_root)

In [13]:
process_transcripts(root_folder, output_root)

In [14]:
sentences_folder = 'Sentences'  # Make sure this points to your 'sentences' directory
documents = []  # Initialize the list to hold document texts

# Walk through the directory and read each text file
for root, dirs, files in os.walk(sentences_folder):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        except UnicodeDecodeError:
            # If a UnicodeDecodeError occurs, try a different encoding or ignore errors
            with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
                text = f.read()
        
        documents.append(text)

# Now 'documents' contains the preprocessed text of each document


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords_tfidf(docs, top_n=10):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
    
    # Get feature names to use as output
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # Extract top n keywords from each document
    keywords = []
    for doc in range(tfidf_matrix.shape[0]):
        scores = tfidf_matrix[doc].toarray().flatten()
        sorted_indices = scores.argsort()[-top_n:][::-1]
        doc_keywords = [feature_names[index] for index in sorted_indices]
        keywords.append(doc_keywords)
    return keywords


In [16]:
from summa import keywords as summa_keywords

def extract_keywords_textrank(text, top_n=10):
    try:
        # Attempt to extract top_n keywords with scores
        tr_keywords = summa_keywords.keywords(text, words=top_n, split=True, scores=True)
        tr_keywords_sorted = sorted(tr_keywords, key=lambda x: x[1], reverse=True)[:top_n]
        return [keyword for keyword, score in tr_keywords_sorted]
    except IndexError:
        # If an IndexError is raised, adjust to return available keywords
        tr_keywords = summa_keywords.keywords(text, split=True, scores=True)
        tr_keywords_sorted = sorted(tr_keywords, key=lambda x: x[1], reverse=True)
        return [keyword for keyword, score in tr_keywords_sorted][:top_n]

In [17]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def extract_keywords_lda(docs, n_topics=5, top_n=10):
    count_vectorizer = CountVectorizer(stop_words='english')
    count_data = count_vectorizer.fit_transform(docs)
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda.fit(count_data)
    
    keywords = []
    words = count_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        top_features_ind = topic.argsort()[-top_n:][::-1]
        topic_keywords = [words[i] for i in top_features_ind]
        keywords.append(topic_keywords)
    return keywords


In [18]:
import nltk
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')

def extract_keywords_ner(text):
    words = word_tokenize(text)
    tagged_words = nltk.pos_tag(words)
    tree = ne_chunk(tagged_words)
    
    return set(' '.join(i[0] for i in t) for t in tree if hasattr(t, 'label') and t.label() in ['PERSON', 'ORGANIZATION', 'GPE', 'LOCATION'])


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/saicharangankidi/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [19]:
sentences_folder = 'Sentences'  # Adjust this to the correct path
documents = []  # This will store the content of each document
file_paths = []  # This will store the path of each document

# Walk through the directory and read each text file
for root, dirs, files in os.walk(sentences_folder):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            documents.append(text)
            file_paths.append(file_path)
        except UnicodeDecodeError:
            # If a UnicodeDecodeError occurs, try reading with a different encoding
            with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
                text = f.read()
            documents.append(text)
            file_paths.append(file_path)

# Now 'documents' contains the content of each document
# and 'file_paths' contains the corresponding file paths


In [20]:
import os

# Assuming 'documents' is a list of texts and 'file_paths' is a list of corresponding file paths
output_root = 'keywords_output'  # Base directory for saving keywords

if not os.path.exists(output_root):
    os.makedirs(output_root)


for file_path, doc in zip(file_paths, documents):
    # Extract keywords with each method
    keywords_tfidf = extract_keywords_tfidf([doc])[0]  # Assuming this function returns a list of lists of keywords
    keywords_textrank = extract_keywords_textrank(doc)  # Assuming this function returns a list of keywords
    keywords_lda = extract_keywords_lda([doc])[0]  # Assuming this function returns a list of lists of keywords for topics
    keywords_ner = list(extract_keywords_ner(doc))  # Converting set to list for uniformity

    # Organize keywords by method
    method_keywords = {
        'tfidf': keywords_tfidf,
        'textrank': keywords_textrank,
        'lda': keywords_lda,
        'ner': keywords_ner
    }

    # Iterate over each method and save the keywords
    for method, keywords in method_keywords.items():
        output_dir = os.path.join(output_root, method_dirs[method])
        os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
        output_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_{method}.txt"
        output_file_path = os.path.join(output_dir, output_file_name)

        # Save keywords to file
        with open(output_file_path, 'w', encoding='utf-8') as f:
            if method == 'lda':  # Special handling for LDA output, which is a list of lists
                for topic_keywords in keywords:
                    f.write(', '.join(topic_keywords) + '\n\n')
            else:
                f.write('\n'.join(keywords) + '\n')


NameError: name 'method_dirs' is not defined

In [ ]:
documents[-1]

In [ ]:
keywords_tfidf

In [ ]:
combined_keywords=keywords_tfidf+keywords_textrank

In [ ]:
combined_keywords

In [ ]:
from collections import defaultdict
import itertools
combined_keywords = list(set(keywords_tfidf + keywords_textrank))
co_occurrences = defaultdict(int)
for doc in documents:
    sentences = nltk.sent_tokenize(doc)
    for sentence in sentences:
        sentence_keywords = [kw for kw in combined_keywords if kw in sentence]
        for pair in itertools.combinations(set(sentence_keywords), 2):
            co_occurrences[pair] += 1

In [ ]:
co_occurrences

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph()
for (kw1, kw2), count in co_occurrences.items():
    G.add_edge(kw1, kw2, weight=count)
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, k=0.5)  # Layout for visualizing the graph's structure
nx.draw_networkx(G, pos, node_size=20, edge_color='gray', alpha=0.5, with_labels=True, font_size=10)
plt.title("Keyword Co-occurrence Graph", size=15)
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph()
for pair, weight in co_occurrences.items():
    G.add_edge(pair[0], pair[1], weight=weight)
pos = nx.spring_layout(G, k=0.1, iterations=50)

# Drawing nodes and edges
nx.draw_networkx_nodes(G, pos, node_size=20, node_color='blue', alpha=0.6)
nx.draw_networkx_edges(G, pos, width=0.25, alpha=1)
nx.draw_networkx_labels(G, pos, font_size=8, font_family='sans-serif')
plt.show()


In [ ]:
degrees = dict(G.degree())
sorted_degrees = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
print("Top 10 central keywords:", sorted_degrees[:10])


In [ ]:
from networkx.algorithms.community import greedy_modularity_communities

communities = list(greedy_modularity_communities(G))
for i, comm in enumerate(communities, 1):
    print(f"Community {i}: ", list(comm)[:10])  # Displaying first 10 keywords of each community


In [ ]:
shortest_path = nx.shortest_path(G, source='good', target='great', weight='weight')
print("Shortest path from 'good' to 'great':", shortest_path)

In [ ]:
# Your shortest path data
shortest_path = ['good', 'reflection', 'goal', 'great']

# Sample text for the fill-in-the-blanks puzzle
text_template = """
1. The journey of self-improvement is always ongoing, striving from being ___1___ to achieving greatness.
2. A moment of ___2___ can sometimes show us the direction we need to take to reach our ___3___.
3. Eventually, with dedication, we reach the ___4___ outcome we aspired for.
"""

# Function to create fill-in-the-blanks puzzle
def create_fill_in_the_blanks(text_template, path_keywords):
    blanks = [f"___{i}___" for i, _ in enumerate(path_keywords, start=1)]
    for blank, keyword in zip(blanks, path_keywords):
        text_template = text_template.replace(blank, "__________")
    return text_template
# Function to ask user to fill in the blanks and check against the shortest path
def solve_fill_in_the_blanks(puzzle_text, path_keywords):
    user_answers = []
    for i, keyword in enumerate(path_keywords, start=1):
        # User input for each blank
        user_input = input(f"Enter the word for blank {i}: ").strip().lower()
        user_answers.append(user_input)
    
    # Validate user answers
    correct_answers = 0
    for user_answer, correct_answer in zip(user_answers, path_keywords):
        if user_answer == correct_answer:
            correct_answers += 1
        else:
            print(f"The correct word for blank was '{correct_answer}', you entered '{user_answer}'.")

    print(f"\nYou got {correct_answers} out of {len(path_keywords)} correct!")

# Create puzzle text
puzzle_text = create_fill_in_the_blanks(text_template, shortest_path)



In [ ]:
print(puzzle_text)

In [ ]:
solve_fill_in_the_blanks(puzzle_text, shortest_path)

In [ ]:
import re

# Assuming 'text' is your preprocessed text with sentences.
text = """
The journey of self-improvement is always ongoing, striving from being good to achieving greatness.
A moment of reflection can sometimes show us the direction we need to take to reach our goal.
Eventually, with dedication, we reach the great outcome we aspired for.
"""

# Keywords from the shortest path to create questions
keywords = ['good', 'reflection', 'goal', 'great']

# Function to replace keywords with blanks in text
def replace_keywords_with_blanks(text, keywords):
    for keyword in keywords:
        text = re.sub(f"\\b{keyword}\\b", '__________', text, flags=re.IGNORECASE)
    return text

# Function to display the puzzle with blanks and then reveal the answers
def display_puzzle_with_answers(text, keywords):
    puzzle_text = replace_keywords_with_blanks(text, keywords)
    print("Fill-in-the-Blanks Puzzle:\n")
    print(puzzle_text)
    print("\nAnswers:")
    for keyword in keywords:
        print(f"The word for the blank is: {keyword}")

# Generate the puzzle
display_puzzle_with_answers(text, keywords)
